<a href="https://colab.research.google.com/github/OlivaVe/project4_team7/blob/main/Are_Credit_Approvals_influenced_by_Demographics%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,061 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,252 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,219 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/main amd64 Packages [50.3 kB]
Get:13 http://archive.ubuntu

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Start using the CSV files. And some ETL

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url1 = "https://aws-project4-team7.s3.amazonaws.com/Data/application_record.csv"
url2 = "https://aws-project4-team7.s3.amazonaws.com/Data/credit_record.csv"
spark.sparkContext.addFile(url1)
df1 = spark.read.csv(SparkFiles.get("application_record.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
spark.sparkContext.addFile(url2)
df2 = spark.read.csv(SparkFiles.get("credit_record.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

In [5]:
# Create a Spark session
spark = SparkSession.builder.appName("SparkToPandas").getOrCreate()

# Assuming you have a Spark DataFrame named "spark_df"
# You can replace this with your actual DataFrame
# spark_df = ...

# Convert the Spark DataFrame to a pandas DataFrame
df = df1.toPandas()
df_cr = df2.toPandas()

# Stop the Spark session when done
spark.stop()

In [6]:
# Reading initial datasets
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,None,2
1,5008805,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,None,2
2,5008806,M,Y,Y,0,112500,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,Security staff,2
3,5008808,F,N,Y,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,Sales staff,1
4,5008809,F,N,Y,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,Sales staff,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,None,1
438553,6840222,F,N,N,0,103500,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,Laborers,1
438554,6841878,F,N,N,0,54000,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,Sales staff,1
438555,6842765,F,N,Y,0,72000,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,None,2


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   438557 non-null  object
 1   CODE_GENDER          438557 non-null  object
 2   FLAG_OWN_CAR         438557 non-null  object
 3   FLAG_OWN_REALTY      438557 non-null  object
 4   CNT_CHILDREN         438557 non-null  object
 5   AMT_INCOME_TOTAL     438557 non-null  object
 6   NAME_INCOME_TYPE     438557 non-null  object
 7   NAME_EDUCATION_TYPE  438557 non-null  object
 8   NAME_FAMILY_STATUS   438557 non-null  object
 9   NAME_HOUSING_TYPE    438557 non-null  object
 10  DAYS_BIRTH           438557 non-null  object
 11  DAYS_EMPLOYED        438557 non-null  object
 12  OCCUPATION_TYPE      304354 non-null  object
 13  CNT_FAM_MEMBERS      438557 non-null  object
dtypes: object(14)
memory usage: 46.8+ MB


In [8]:
# Reading initial datasets
df_cr

,ID,STATUS
0,5001711,1
1,5001712,1
2,5001713,1
3,5001714,1
4,5001715,1
...,...,...
33851,5150481,1
33852,5150483,1
33853,5150484,1
33854,5150485,1


In [9]:
df_cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33856 entries, 0 to 33855
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      33856 non-null  object
 1   STATUS  33856 non-null  object
dtypes: object(2)
memory usage: 529.1+ KB


In [10]:
df.head(3) # it has 438557 samples

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,None,2
1,5008805,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,None,2
2,5008806,M,Y,Y,0,112500,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,Security staff,2


In [11]:
df = df.drop(columns='OCCUPATION_TYPE')
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2
1,5008805,M,Y,Y,0,427500,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,2
2,5008806,M,Y,Y,0,112500,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,2
3,5008808,F,N,Y,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1
4,5008809,F,N,Y,0,270000,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1
438553,6840222,F,N,N,0,103500,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1
438554,6841878,F,N,N,0,54000,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1
438555,6842765,F,N,Y,0,72000,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,2


# Creating only one dataframe.

In [12]:
# Merging the 2 datasets with column ID. Reduced to 24320 samples.
df_t = df.merge(df_cr, how='inner', on='ID')
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24672 entries, 0 to 24671
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   24672 non-null  object
 1   CODE_GENDER          24672 non-null  object
 2   FLAG_OWN_CAR         24672 non-null  object
 3   FLAG_OWN_REALTY      24672 non-null  object
 4   CNT_CHILDREN         24672 non-null  object
 5   AMT_INCOME_TOTAL     24672 non-null  object
 6   NAME_INCOME_TYPE     24672 non-null  object
 7   NAME_EDUCATION_TYPE  24672 non-null  object
 8   NAME_FAMILY_STATUS   24672 non-null  object
 9   NAME_HOUSING_TYPE    24672 non-null  object
 10  DAYS_BIRTH           24672 non-null  object
 11  DAYS_EMPLOYED        24672 non-null  object
 12  CNT_FAM_MEMBERS      24672 non-null  object
 13  STATUS               24672 non-null  object
dtypes: object(14)
memory usage: 2.8+ MB


In [13]:
df_t['CNT_CHILDREN'] = df_t['CNT_CHILDREN'].astype(float)
df_t['AMT_INCOME_TOTAL'] = df_t['AMT_INCOME_TOTAL'].astype(float)
df_t['DAYS_BIRTH'] = df_t['DAYS_BIRTH'].astype(float)
df_t['DAYS_EMPLOYED'] = df_t['DAYS_EMPLOYED'].astype(float)
df_t['CNT_FAM_MEMBERS'] = df_t['CNT_FAM_MEMBERS'].astype(float)
df_t['STATUS'] = df_t['STATUS'].astype(float)

In [14]:
# Dropping useless column
df_t =df_t.drop(columns='ID')
df_t.head(3)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,STATUS
0,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005.0,-4542.0,2.0,1.0
1,M,Y,Y,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005.0,-4542.0,2.0,1.0
2,M,Y,Y,0.0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474.0,-1134.0,2.0,1.0


In [15]:
#Changing in 'FLAG_OWN_CAR' to 1 and 0 for easier use.
for index, row in df_t.iterrows():
    if row['FLAG_OWN_CAR'] == 'N':
        df_t.at[index, 'FLAG_OWN_CAR'] = 0
    elif row['FLAG_OWN_CAR'] == 'Y':
        df_t.at[index, 'FLAG_OWN_CAR'] = 1
#Changing in 'FLAG_OWN_REALTY' to 1 and 0 for easier use.
for index, row in df_t.iterrows():
    if row['FLAG_OWN_REALTY'] == 'N':
        df_t.at[index, 'FLAG_OWN_REALTY'] = 0
    elif row['FLAG_OWN_REALTY'] == 'Y':
        df_t.at[index, 'FLAG_OWN_REALTY'] = 1

In [16]:
#Visualizing the changes
[df_t['FLAG_OWN_CAR'].unique(), df_t['FLAG_OWN_REALTY'].unique()]

[array([1, 0], dtype=object), array([1, 0], dtype=object)]

## General study of some of our variables

In [17]:
#Visualizing variables
df_t['CNT_CHILDREN'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5., 14., 19.])

In [18]:
#Counting the number of variables
for x in range(0, 20):
  print(x,'-',(df_t[df_t['CNT_CHILDREN']==x]['DAYS_BIRTH']/365).round().count())

0 - 17323
1 - 4852
2 - 2150
3 - 295
4 - 38
5 - 10
6 - 0
7 - 0
8 - 0
9 - 0
10 - 0
11 - 0
12 - 0
13 - 0
14 - 3
15 - 0
16 - 0
17 - 0
18 - 0
19 - 1


In [19]:
#Cleaning the data
for x in range(6, 20):
  df_t= df_t[df_t['CNT_CHILDREN'] != x]
#Showing the new result of variables
df_t['CNT_CHILDREN'].unique()

array([0., 1., 2., 3., 4., 5.])

In [20]:
# df.shape[1]
for x in (5, 6, 7,8,11,12):
  print(df_t.columns[x], '-', df_t.iloc[:, x].unique(),'\n')

NAME_INCOME_TYPE - ['Working' 'Commercial associate' 'Pensioner' 'State servant' 'Student'] 

NAME_EDUCATION_TYPE - ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree'] 

NAME_FAMILY_STATUS - ['Civil marriage' 'Married' 'Single / not married' 'Separated' 'Widow'] 

NAME_HOUSING_TYPE - ['Rented apartment' 'House / apartment' 'Municipal apartment'
 'With parents' 'Co-op apartment' 'Office apartment'] 

CNT_FAM_MEMBERS - [2. 1. 3. 4. 5. 6. 7.] 

STATUS - [1. 0.] 



In [21]:
# Applications grouped by family members

for x in range(1, 8):
  print(x,'-',(df_t[df_t['CNT_FAM_MEMBERS']==x]['DAYS_BIRTH']/365).round().count())

1 - 4825
2 - 13279
3 - 4196
4 - 2041
5 - 281
6 - 37
7 - 9


In [22]:
# Transforming column DAYS BIRTH into Age in years
for index, row in df_t.iterrows():
  df_t.at[index, 'DAYS_BIRTH'] = ((df_t.at[index, 'DAYS_BIRTH']*-1)/365.25).round()

In [23]:
# Transforming column DAYS EMployed into Worked years
for index, row in df_t.iterrows():
  df_t.at[index, 'DAYS_EMPLOYED'] = ((df_t.at[index, 'DAYS_EMPLOYED']*-1)/365.25).round()

In [24]:
# Double checking our transformation
[df_t['DAYS_BIRTH'].unique(), df_t['DAYS_EMPLOYED'].unique()]

[array([33., 59., 52., 62., 46., 49., 29., 27., 30., 32., 56., 43., 44.,
        34., 55., 47., 42., 37., 28., 61., 57., 45., 54., 40., 51., 39.,
        48., 24., 21., 35., 36., 26., 50., 41., 58., 60., 63., 31., 38.,
        53., 22., 64., 23., 25., 65., 67., 66., 69., 68.]),
 array([   12.,     3.,     8., -1000.,     2.,     4.,     6.,     9.,
           19.,    15.,     5.,    13.,     7.,    17.,    30.,    16.,
           11.,     1.,    10.,    24.,    14.,    25.,     0.,    20.,
           27.,    18.,    21.,    23.,    22.,    28.,    39.,    32.,
           33.,    37.,    26.,    38.,    31.,    29.,    35.,    34.,
           41.,    40.,    36.,    42.,    43.])]

In [25]:
# Transforming Retired citizens data so that it makes sense
for index, row in df_t.iterrows():
  if row['DAYS_EMPLOYED'] == -1000:
    df_t.at[index, 'DAYS_EMPLOYED'] = 45

df_t['DAYS_EMPLOYED'].value_counts()

45.0    4133
1.0     2226
2.0     2028
3.0     1968
4.0     1853
5.0     1609
6.0     1389
7.0     1333
8.0     1129
9.0     1066
10.0     810
11.0     621
0.0      619
15.0     503
12.0     495
13.0     451
14.0     333
20.0     235
16.0     220
17.0     201
18.0     176
19.0     171
21.0     157
22.0     136
23.0     110
24.0      94
26.0      84
27.0      81
25.0      80
28.0      71
29.0      44
30.0      41
33.0      33
32.0      29
31.0      27
35.0      24
41.0      24
34.0      21
36.0      13
40.0      12
38.0       7
39.0       4
37.0       3
43.0       3
42.0       1
Name: DAYS_EMPLOYED, dtype: int64

In [26]:
# Renaming columns
df_t.rename(
    columns={"CODE_GENDER": "GENDER", "FLAG_OWN_REALTY": "FLAG_OWN_PROPERTY", "NAME_INCOME_TYPE": "INCOME_TYPE", "NAME_EDUCATION_TYPE": "EDUCATION_TYPE", "NAME_FAMILY_STATUS": "FAMILY_STATUS", "NAME_HOUSING_TYPE": "HOUSING_TYPE", "DAYS_BIRTH": "AGE", "DAYS_EMPLOYED": "YEARS_EMPLOYED", "DAYS_EMPLOYED": "YEARS_EMPLOYED", "STATUS": "APPROVAL_STATUS"},
    inplace=True,
)
df_t.head(1)

,GENDER,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,INCOME_TYPE,EDUCATION_TYPE,FAMILY_STATUS,HOUSING_TYPE,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS
0,M,1,1,0.0,427500.0,Working,Higher education,Civil marriage,Rented apartment,33.0,12.0,2.0,1.0


In [27]:
# saving the CSV for Tableau

df_t.to_csv('/content/sample_data/application_record_revised_tableau.csv')

In [28]:
# Saving to read with spark
df_t.to_csv('/content/sample_data/application_record_revised.csv')

In [29]:
# Choosing categorical columns that need to be transformed into dummy variables
columns_to_convert = [
    'GENDER', 'INCOME_TYPE', 'EDUCATION_TYPE',
    'FAMILY_STATUS', 'HOUSING_TYPE']
dummies = pd.get_dummies(df_t, columns=columns_to_convert)

In [30]:
# dummies.columns
dummies

,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS,GENDER_F,GENDER_M,...,FAMILY_STATUS_Married,FAMILY_STATUS_Separated,FAMILY_STATUS_Single / not married,FAMILY_STATUS_Widow,HOUSING_TYPE_Co-op apartment,HOUSING_TYPE_House / apartment,HOUSING_TYPE_Municipal apartment,HOUSING_TYPE_Office apartment,HOUSING_TYPE_Rented apartment,HOUSING_TYPE_With parents
0,1,1,0.0,427500.0,33.0,12.0,2.0,1.0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,1,0.0,427500.0,33.0,12.0,2.0,1.0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,1,0.0,112500.0,59.0,3.0,2.0,1.0,0,1,...,1,0,0,0,0,1,0,0,0,0
3,0,1,0.0,270000.0,52.0,8.0,1.0,1.0,1,0,...,0,0,1,0,0,1,0,0,0,0
4,0,1,0.0,270000.0,52.0,8.0,1.0,1.0,1,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24667,1,1,0.0,315000.0,47.0,7.0,2.0,0.0,0,1,...,1,0,0,0,0,1,0,0,0,0
24668,0,1,0.0,157500.0,34.0,4.0,2.0,1.0,1,0,...,1,0,0,0,0,1,0,0,0,0
24669,0,1,0.0,157500.0,34.0,4.0,2.0,1.0,1,0,...,1,0,0,0,0,1,0,0,0,0
24670,0,1,0.0,283500.0,49.0,2.0,2.0,0.0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [32]:
# Split our preprocessed data into our features and target arrays
X = dummies.drop(columns='APPROVAL_STATUS')
y = dummies['APPROVAL_STATUS']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [33]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Trainig the model

In [34]:
import tensorflow as tf

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=30, activation='relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2480      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 9,041
Trainable params: 9,041
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [37]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 3s 2ms/step - loss: 0.0783 - accuracy: 0.9868
Epoch 2/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0679 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0665 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0656 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0649 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0642 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0636 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 2s 4ms/step - loss: 0.0625 - accuracy: 0.9876
Epoch 9/10
579/579 [==============================] - 1s 3ms/step - loss: 0.0623 - accuracy: 0.9875
Epoch 10/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0611 - accuracy: 0.9873

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 0s - loss: 0.0835 - accuracy: 0.9849 - 449ms/epoch - 2ms/step
Loss: 0.08346722275018692, Accuracy: 0.9849197268486023


---
Defining important features

We will use a Random Forest to see which features are the most miningful, and reduce the amount of features in the model
---

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
X = dummies.drop(columns='APPROVAL_STATUS')
y = dummies['APPROVAL_STATUS']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=y)


# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Model -  RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(X_train_scaled, y_train)
print(forest.score(X_test_scaled, y_test))

pd.DataFrame({
    'feature': X.columns,
    'importance': forest.feature_importances_
}).sort_values(by='importance', ascending=False)

0.9860548078482244


,feature,importance
4,AGE,0.261402
3,AMT_INCOME_TOTAL,0.241768
5,YEARS_EMPLOYED,0.154060
1,FLAG_OWN_PROPERTY,0.040484
6,CNT_FAM_MEMBERS,0.038621
0,FLAG_OWN_CAR,0.034008
2,CNT_CHILDREN,0.027355
9,INCOME_TYPE_Commercial associate,0.019425
13,INCOME_TYPE_Working,0.018335
15,EDUCATION_TYPE_Higher education,0.016042


----
# Second Model
-----

In [40]:
# We defined that the following columns are not that important, so we eliminate them
df_p= df_t.drop(columns = [
    'GENDER', 'INCOME_TYPE', 'EDUCATION_TYPE',
    'FAMILY_STATUS', 'HOUSING_TYPE'])
df_p

,FLAG_OWN_CAR,FLAG_OWN_PROPERTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AGE,YEARS_EMPLOYED,CNT_FAM_MEMBERS,APPROVAL_STATUS
0,1,1,0.0,427500.0,33.0,12.0,2.0,1.0
1,1,1,0.0,427500.0,33.0,12.0,2.0,1.0
2,1,1,0.0,112500.0,59.0,3.0,2.0,1.0
3,0,1,0.0,270000.0,52.0,8.0,1.0,1.0
4,0,1,0.0,270000.0,52.0,8.0,1.0,1.0
...,...,...,...,...,...,...,...,...
24667,1,1,0.0,315000.0,47.0,7.0,2.0,0.0
24668,0,1,0.0,157500.0,34.0,4.0,2.0,1.0
24669,0,1,0.0,157500.0,34.0,4.0,2.0,1.0
24670,0,1,0.0,283500.0,49.0,2.0,2.0,0.0


In [41]:
# Split our preprocessed data into our features and target arrays
X = df_p.drop(columns='APPROVAL_STATUS')
y = df_p['APPROVAL_STATUS']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [42]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
scaler.fit(X_train)
# Scale the data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn2 = tf.keras.models.Sequential()
# First hidden layer
nn2.add(tf.keras.layers.Dense(units=80, input_dim=7, activation='relu'))
# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=80, activation='relu'))
# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                640       
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 1)                 81        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn2.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [45]:
# Train the model
nn2.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 3s 3ms/step - loss: 0.0943 - accuracy: 0.9792
Epoch 2/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0710 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0700 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 2s 4ms/step - loss: 0.0693 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0690 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0683 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0682 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0680 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0678 - accuracy: 0.9874
Epoch 10/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0676 - accuracy: 0.9874

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 0s - loss: 0.0793 - accuracy: 0.9851 - 436ms/epoch - 2ms/step
Loss: 0.07925745844841003, Accuracy: 0.9850819110870361


----
# 3rd model
---

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()
# First hidden layer
nn3.add(tf.keras.layers.Dense(units=100, input_dim=7, activation='tanh'))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=100, activation='tanh'))
nn3.add(tf.keras.layers.Dense(units=100, activation='tanh'))
# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               800       
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dense_8 (Dense)             (None, 100)               10100     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21,101
Trainable params: 21,101
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the model
nn3.compile(
    loss = 'binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy']
)

In [49]:
# Train the model
nn3.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 3s 3ms/step - loss: 0.1003 - accuracy: 0.9799
Epoch 2/10
579/579 [==============================] - 2s 4ms/step - loss: 0.0687 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0688 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 1s 3ms/step - loss: 0.0689 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0682 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0681 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 1s 3ms/step - loss: 0.0682 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0681 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0678 - accuracy: 0.9874
Epoch 10/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0682 - accuracy: 0.9874

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 1s - loss: 0.0790 - accuracy: 0.9851 - 666ms/epoch - 3ms/step
Loss: 0.07898065447807312, Accuracy: 0.9850819110870361


----
# 4th model

In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn3 = tf.keras.models.Sequential()
# First hidden layer
nn3.add(tf.keras.layers.Dense(units=100, input_dim=7, activation='tanh'))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               800       
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 11,001
Trainable params: 11,001
Non-trainable params: 0
_________________________________________________________________


In [52]:
# Compile the model
nn3.compile(
    loss = 'binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy']
)

In [53]:
# Train the model
nn3.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
579/579 [==============================] - 2s 2ms/step - loss: 0.0966 - accuracy: 0.9858
Epoch 2/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0707 - accuracy: 0.9874
Epoch 3/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0695 - accuracy: 0.9874
Epoch 4/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0690 - accuracy: 0.9874
Epoch 5/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0682 - accuracy: 0.9874
Epoch 6/10
579/579 [==============================] - 2s 3ms/step - loss: 0.0677 - accuracy: 0.9874
Epoch 7/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0677 - accuracy: 0.9874
Epoch 8/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0676 - accuracy: 0.9874
Epoch 9/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0674 - accuracy: 0.9874
Epoch 10/10
579/579 [==============================] - 1s 2ms/step - loss: 0.0672 - accuracy: 0.9874

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

193/193 - 1s - loss: 0.0784 - accuracy: 0.9851 - 651ms/epoch - 3ms/step
Loss: 0.07842839509248734, Accuracy: 0.9850819110870361


# Using Spark

In [55]:
# Import packages
from pyspark.sql import SparkSession
from pyspark import SparkFiles
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [56]:
df_revised = spark.read.csv(SparkFiles.get("/content/sample_data/application_record_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

In [57]:
df_revised.show()

+---+------+------------+-----------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----+--------------+---------------+---------------+
|_c0|GENDER|FLAG_OWN_CAR|FLAG_OWN_PROPERTY|CNT_CHILDREN|AMT_INCOME_TOTAL|         INCOME_TYPE|      EDUCATION_TYPE|       FAMILY_STATUS|     HOUSING_TYPE| AGE|YEARS_EMPLOYED|CNT_FAM_MEMBERS|APPROVAL_STATUS|
+---+------+------------+-----------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----+--------------+---------------+---------------+
|  0|     M|           1|                1|         0.0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|33.0|          12.0|            2.0|            1.0|
|  1|     M|           1|                1|         0.0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|33.0|          12.

In [58]:
df_revised.createOrReplaceTempView('temporary_view')

In [59]:
# We wanted to see the quantities of approved credit by Education Type
spark.sql('''
SELECT EDUCATION_TYPE, count(CNT_CHILDREN) AS `APPROVED CREDITS `
FROM temporary_view
WHERE APPROVAL_STATUS = 1
GROUP BY EDUCATION_TYPE
ORDER BY count(CNT_CHILDREN) DESC
''').show()

+--------------------+-----------------+
|      EDUCATION_TYPE|APPROVED CREDITS |
+--------------------+-----------------+
|Secondary / secon...|            16552|
|    Higher education|             6574|
|   Incomplete higher|              947|
|     Lower secondary|              247|
|     Academic degree|               23|
+--------------------+-----------------+



In [60]:
# We wanted to see the quantities of approved credit by Housing Type
spark.sql('''
SELECT HOUSING_TYPE, count(CNT_CHILDREN) AS `APPROVED CREDITS `
FROM temporary_view
WHERE APPROVAL_STATUS = 1
GROUP BY HOUSING_TYPE
ORDER BY count(CNT_CHILDREN) DESC
''').show()

+-------------------+-----------------+
|       HOUSING_TYPE|APPROVED CREDITS |
+-------------------+-----------------+
|  House / apartment|            21728|
|       With parents|             1187|
|Municipal apartment|              771|
|   Rented apartment|              371|
|   Office apartment|              179|
|    Co-op apartment|              107|
+-------------------+-----------------+



In [61]:
# We wanted to see the average of years employed a person hasat the moment of getting a credit, divided by gender.
spark.sql('''
SELECT GENDER, AVG(YEARS_EMPLOYED)
FROM temporary_view
GROUP By GENDER
''').show()

+------+-------------------+
|GENDER|avg(YEARS_EMPLOYED)|
+------+-------------------+
|     F| 15.865955501371532|
|     M|   9.15127677598935|
+------+-------------------+

